<a href="https://colab.research.google.com/github/ajagota7/Reward-Shaping/blob/main/gridworld_ope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Environment

In [1]:
class GridWorld:
    def __init__(self, height, width, start, end, bad_regions, good_regions):
        self.height = height
        self.width = width
        self.start = start
        self.end = end
        self.bad_regions = bad_regions
        self.good_regions = good_regions

    def reset(self):
        self.agent_position = self.start

    def step(self, action):
        x, y = self.agent_position

        if action == "up" and y < self.height - 1:
            y += 1
        elif action == "down" and y > 0:
            y -= 1
        elif action == "left" and x > 0:
            x -= 1
        elif action == "right" and x < self.width - 1:
            x += 1

        self.agent_position = (x, y)

        if self.agent_position == self.end:
            reward = 3
            done = True
        elif self.agent_position in self.bad_regions:
            reward = -1
            done = False
        elif self.agent_position in self.good_regions:
            reward = 0.5
            done = False
        else:
            reward = 0
            done = False

        return (x, y), reward, done


In [2]:
import numpy as np

class Agent:
    def __init__(self, epsilon=0.0):
        self.epsilon = epsilon

    def select_action(self, policy_func):
        if np.random.uniform() < self.epsilon:
            # Choose a random action
            action = np.random.choice(["up", "down", "left", "right"])
        else:
            # Use the provided policy function to get the best action
            action = policy_func()
        return action

# Define different policy functions outside the class

def random_policy():
    # Choose a random action
    return np.random.choice(["up", "down", "left", "right"])

def behavior_policy():
    action_probs = {"up": 0.25, "down": 0.25, "left": 0.25, "right": 0.25}
    return np.random.choice(list(action_probs.keys()), p=list(action_probs.values()))

def evaluation_policy():
    action_probs = {"up": 0.4, "down": 0.1, "left": 0.1, "right": 0.4}
    return np.random.choice(list(action_probs.keys()), p=list(action_probs.values()))

def manhattan_distance(pos1, pos2):
    # Compute the Manhattan distance between two positions
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

# Generating Policy data

In [3]:
# Gridworld environment
height = 5
width  = 5
start = (0,0)
end = (4,4)

In [31]:

env = GridWorld(height, width, start, end, [(1, 1), (2, 2)], [(3,3)])

# Number of episodes
num_episodes = 200

def create_policy_set(env, policy, num_episodes):
  # Create a list to store policies as trajectories
  policies = []

  # Run multiple episodes
  for episode in range(num_episodes):
      # Create a new Agent for each episode to generate a different policy
      agent = Agent(epsilon=0.0)

      # Run an episode
      env.reset()
      done = False
      trajectory = []  # Store the trajectory for the current episode
      cumulative_reward = 0.0  # Initialize cumulative reward
      while not done:
          state = env.agent_position  # Get the current state
          action = agent.select_action(policy)
          next_state, reward, done = env.step(action)

          # Compute cumulative reward
          cumulative_reward += reward

          # Compute feature function values (manhattan distances)
          good_region_distances = [manhattan_distance(state, gr) for gr in env.good_regions]
          bad_region_distances = [manhattan_distance(state, br) for br in env.bad_regions]

          # Store the (state, action, reward, next_state) tuple in the trajectory
          trajectory.append((state, action, reward, next_state, cumulative_reward, good_region_distances, bad_region_distances))

      # Append the trajectory to the policies list
      policies.append(trajectory)

  good_regions_str = "_".join([f"gr_{pos[0]}_{pos[1]}" for pos in env.good_regions])
  bad_regions_str = "_".join([f"br_{pos[0]}_{pos[1]}" for pos in env.bad_regions])
  filename = f"{env.__class__.__name__}_policy_{policy.__name__}_{good_regions_str}_{bad_regions_str}.txt"

  return policies, filename


In [16]:
def calc_V_pi_e(evaluation_policies):

  total_cumulative_reward = 0.0

  for episode_trajectory in evaluation_policies:
      total_cumulative_reward += episode_trajectory[-1][4]

  return total_cumulative_reward/len(evaluation_policies)

In [32]:
behavior_policies, a = create_policy_set(env, behavior_policy, 200)

In [33]:
a

'GridWorld_policy_behavior_policy_gr_3_3_br_1_1_br_2_2.txt'

In [17]:
evaluation_policies = create_policy_set(env, evaluation_policy, 1000)
V_pi_e = calc_V_pi_e(evaluation_policies)

# Saving and Loading Data

In [ ]:
import pickle

def save_policies_to_file(policies, filename):
    with open(filename, 'wb') as file:
        pickle.dump(policies, file)

def load_policies_from_file(filename):
    with open(filename, 'rb') as file:
        policies = pickle.load(file)
    return policies


# Training Reward Models

## State -> Reward

Training model to predict rewards based on state only

In [ ]:
import tensorflow as tf
import numpy as np

# Step 1: Prepare the data
# behavior_policies = [...]  # Replace [...] with your actual behavior_policies list

# Initialize lists to store all 'next_state' and 'reward' values
all_next_states = []
all_rewards = []

# Extract the 'next_state' and 'reward' from the 'behavior_policies' list
for trajectory in behavior_policies:
    # For each trajectory, extract all 'next_state' and 'reward' values
    next_states = [state_action_reward[3] for state_action_reward in trajectory]
    rewards = [state_action_reward[2] for state_action_reward in trajectory]

    # Append the values to the corresponding lists
    all_next_states.extend(next_states)
    all_rewards.extend(rewards)

# Convert 'next_states' and 'rewards' into appropriate formats for training
all_next_states = np.array(all_next_states, dtype=np.float32)
all_rewards = np.array(all_rewards, dtype=np.float32)

# Now, 'all_next_states' contains all the 'next_state' values, and 'all_rewards' contains all the corresponding rewards.


In [ ]:
len(all_rewards)

21499

In [ ]:
# Step 2: Design the neural network
class RewardPredictorStates(tf.keras.Model):
    def __init__(self, input_shape):
        super(RewardPredictorStates, self).__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return x

# Define hyperparameters for the neural network
input_shape = all_next_states.shape[1:]  # Shape of the input state (excluding batch size)
learning_rate = 0.005
num_epochs = 1000
batch_size = 32

# Create the neural network
reward_predictor_states = RewardPredictorStates(input_shape)

# Compile the model
reward_predictor_states.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                         loss='mean_squared_error')

# Step 3: Train the neural network with early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
]

reward_predictor_states.fit(
    all_next_states, all_rewards,
    batch_size=batch_size, epochs=num_epochs,
    callbacks=callbacks, validation_split=0.2, verbose=1
)

Epoch 1/1000


KeyboardInterrupt: ignored

In [ ]:
current_state = behavior_policies[0][5][0]
current_state = np.array(current_state, dtype=np.float32)
predicted_reward_current = reward_predictor_states.predict(np.expand_dims(current_state, axis=0))
print("Predicted Reward Current State:", predicted_reward_current[0, 0])

new_state = behavior_policies[0][5][3]  # Replace ... with the new state for which you want to predict the reward
new_state = np.array(new_state, dtype=np.float32)
predicted_reward_next = reward_predictor_states.predict(np.expand_dims(new_state, axis=0))
print("Predicted Reward Next State:", predicted_reward_next[0, 0])

1/1 [==============================] - 0s 20ms/step
Predicted Reward Current State: -0.03829813
1/1 [==============================] - 0s 20ms/step
Predicted Reward Next State: -0.006269932


## State -> Cumulative Reward

Reward model based on State -> Cumulative Rewards

In [ ]:
import tensorflow as tf
import numpy as np

# Step 1: Prepare the data
# behavior_policies = [...]  # Replace [...] with your actual behavior_policies list

# Initialize lists to store all 'next_state' and 'reward' values
all_next_states = []
all_cum_rewards = []

# Extract the 'next_state' and 'reward' from the 'behavior_policies' list
for trajectory in behavior_policies:
    # For each trajectory, extract all 'next_state' and 'reward' values
    next_states = [state_action_reward[3] for state_action_reward in trajectory]
    cum_rewards = [state_action_reward[4] for state_action_reward in trajectory]

    # Append the values to the corresponding lists
    all_next_states.extend(next_states)
    all_cum_rewards.extend(cum_rewards)

# Convert 'next_states' and 'rewards' into appropriate formats for training
all_next_states = np.array(all_next_states, dtype=np.float32)
all_cum_rewards = np.array(all_cum_rewards, dtype=np.float32)

# Now, 'all_next_states' contains all the 'next_state' values, and 'all_rewards' contains all the corresponding rewards.


In [ ]:
len(all_cum_rewards)

21499

In [ ]:
# Step 2: Design the neural network
# import tensorflow as tf

class RewardPredictorCumulative(tf.keras.Model):
    def __init__(self, input_shape):
        super(RewardPredictorCumulative, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.batch_norm1 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.batch_norm2 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(1, kernel_initializer='he_normal')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.batch_norm1(x)
        x = self.dense2(x)
        x = self.batch_norm2(x)
        x = self.dense3(x)
        return x


# Define hyperparameters for the neural network
input_shape = all_next_states.shape[1:]  # Shape of the input state (excluding batch size)
learning_rate = 0.0001
num_epochs = 1000
batch_size = 32

# Create the neural network
reward_predictor_cumulative = RewardPredictorCumulative(input_shape)

# Compile the model
reward_predictor_cumulative.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                         loss='mean_squared_error')

# Step 3: Train the neural network with early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
]

reward_predictor_cumulative.fit(
    all_next_states, all_cum_rewards,
    batch_size=batch_size, epochs=num_epochs,
    callbacks=callbacks, validation_split=0.2, verbose=1
)

Epoch 1/1000
2691/2691 [==============================] - 14s 4ms/step - loss: 14287.8623 - val_loss: 9193.1582
Epoch 2/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 12563.5850 - val_loss: 7138.1450
Epoch 3/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 10212.1973 - val_loss: 5365.0083
Epoch 4/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 8540.7607 - val_loss: 4638.1211
Epoch 5/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 7985.4175 - val_loss: 4635.2275
Epoch 6/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 7930.9414 - val_loss: 4674.1807
Epoch 7/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 7929.3970 - val_loss: 4667.2554
Epoch 8/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 7929.0581 - val_loss: 4669.0732
Epoch 9/1000
2691/2691 [==============================] - 12s 4ms/step - loss: 7927.9824 - val_loss: 4675.181

## State -> Rewards over past 3 timesteps

Cumulative rewards over 3 timesteps

In [ ]:
# Discounted sum
# Create a new list to store trajectories with the new data
augmented_behavior_policies = []

# Set the discount factor (gamma)
discount_factor = 0.9  # You can adjust this value as needed (usually between 0 and 1)

# Iterate through each trajectory in behavior_policies
for trajectory in behavior_policies:
    num_timesteps = len(trajectory)
    new_trajectory = []

    # Iterate through each timestep in the trajectory
    for t in range(num_timesteps):
        # Calculate the discounted sum of the past 3 rewards for the past 3 timesteps
        discounted_sum = 0.0
        for i in range(1, min(4, t + 1)):
            discounted_sum += (discount_factor ** (i - 1)) * trajectory[t - i][2]

        # Update the trajectory to include only the discounted sum of the past 3 rewards
        state, action, reward, next_state, cumulative_reward, good_prox, bad_prox = trajectory[t]
        new_trajectory.append((state, discounted_sum, action, reward, next_state, cumulative_reward))

    # Append the modified trajectory to the augmented_behavior_policies list
    augmented_behavior_policies.append(new_trajectory)


In [ ]:
# Discounted sum
# Create a new list to store trajectories with the new data
augmented_evaluation_policies = []

# Set the discount factor (gamma)
discount_factor = 0.9  # You can adjust this value as needed (usually between 0 and 1)

# Iterate through each trajectory in evaluation_policies
for trajectory in evaluation_policies:
    num_timesteps = len(trajectory)
    new_trajectory = []

    # Iterate through each timestep in the trajectory
    for t in range(num_timesteps):
        # Calculate the discounted sum of the past 3 rewards for the past 3 timesteps
        discounted_sum = 0.0
        for i in range(1, min(4, t + 1)):
            discounted_sum += (discount_factor ** (i - 1)) * trajectory[t - i][2]

        # Update the trajectory to include only the discounted sum of the past 3 rewards
        state, action, reward, next_state, cumulative_reward, good_prox. bad_prox = trajectory[t]
        new_trajectory.append((state, discounted_sum, action, reward, next_state, cumulative_reward))

    # Append the modified trajectory to the augmented_evaluation_policies list
    augmented_evaluation_policies.append(new_trajectory)


NameError: ignored

In [ ]:
import tensorflow as tf
import numpy as np

# Step 1: Prepare the data
def preprocess_nstep_data(policy_data):
  # Initialize lists to store all 'next_state' and 'reward' values
  all_next_states = []
  all_past3_rewards = []

  # Extract the 'next_state' and 'reward' from the 'behavior_policies' list
  for trajectory in policy_data:
      # For each trajectory, extract all 'next_state' and 'reward' values
      next_states = [state_action_reward[4] for state_action_reward in trajectory]
      rewards = [state_action_reward[3] for state_action_reward in trajectory]

      # Append the values to the corresponding lists
      all_next_states.extend(next_states)
      all_past3_rewards.extend(rewards)

  # Convert 'next_states' and 'rewards' into appropriate formats for training
  all_next_states = np.array(all_next_states, dtype=np.float32)
  all_past3_rewards = np.array(all_past3_rewards, dtype=np.float32)
  return all_next_states, all_past3_rewards

In [ ]:
# Step 2: Design the neural network
class RewardPredictor3States(tf.keras.Model):
    def __init__(self, input_shape):
        super(RewardPredictor3States, self).__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return x


In [ ]:
# Behavior policies training
all_next_states_behav, all_past3_rewards_behav = preprocess_nstep_data(augmented_behavior_policies)

# Define hyperparameters for the neural network
input_shape = all_next_states_behav.shape[1:]  # Shape of the input state (excluding batch size)
learning_rate = 0.005
num_epochs = 1000
batch_size = 32

# Create the neural network
reward_predictor_3states = RewardPredictor3States(input_shape)

# Compile the model
reward_predictor_3states.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                         loss='mean_squared_error')

# Step 3: Train the neural network with early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
]

reward_predictor_3states.fit(
    all_next_states_behav, all_past3_rewards_behav,
    batch_size=batch_size, epochs=num_epochs,
    callbacks=callbacks, validation_split=0.2, verbose=1
)

Epoch 1/1000
541/541 [==============================] - 2s 2ms/step - loss: 0.1163 - val_loss: 0.0807
Epoch 2/1000
541/541 [==============================] - 1s 2ms/step - loss: 0.0498 - val_loss: 0.0302
Epoch 3/1000
541/541 [==============================] - 1s 2ms/step - loss: 0.0184 - val_loss: 0.0124
Epoch 4/1000
541/541 [==============================] - 1s 2ms/step - loss: 0.0085 - val_loss: 0.0050
Epoch 5/1000
541/541 [==============================] - 1s 2ms/step - loss: 0.0036 - val_loss: 0.0026
Epoch 6/1000
541/541 [==============================] - 1s 2ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 7/1000
541/541 [==============================] - 1s 2ms/step - loss: 8.8721e-04 - val_loss: 6.9921e-04
Epoch 8/1000
541/541 [==============================] - 1s 2ms/step - loss: 5.3470e-04 - val_loss: 2.7174e-04
Epoch 9/1000
541/541 [==============================] - 1s 2ms/step - loss: 3.5016e-04 - val_loss: 4.5576e-04
Epoch 10/1000
541/541 [==============================] - 1

In [ ]:
current_state = [2,3]
current_state = np.array(current_state, dtype=np.float32)
predicted_reward_current = reward_predictor_3states.predict(np.expand_dims(current_state, axis=0))
print("Predicted Reward Current State:", predicted_reward_current[0, 0])

1/1 [==============================] - 0s 23ms/step
Predicted Reward Current State: 0.0006133178


In [ ]:
# Evaluation policies training
all_next_states_eval, all_past3_rewards_eval = preprocess_nstep_data(augmented_evaluation_policies)

# Define hyperparameters for the neural network
input_shape = all_next_states_eval.shape[1:]  # Shape of the input state (excluding batch size)
learning_rate = 0.005
num_epochs = 1000
batch_size = 32

# Create the neural network
reward_predictor_3states_eval = RewardPredictor3States(input_shape)

# Compile the model
reward_predictor_3states_eval.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                         loss='mean_squared_error')

# Step 3: Train the neural network with early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
]

reward_predictor_3states_eval.fit(
    all_next_states_eval, all_past3_rewards_eval,
    batch_size=batch_size, epochs=num_epochs,
    callbacks=callbacks, validation_split=0.2, verbose=1
)

Epoch 1/1000
83/83 [==============================] - 1s 4ms/step - loss: 0.4849 - val_loss: 0.4438
Epoch 2/1000
83/83 [==============================] - 0s 2ms/step - loss: 0.4098 - val_loss: 0.4183
Epoch 3/1000
83/83 [==============================] - 0s 2ms/step - loss: 0.3815 - val_loss: 0.4099
Epoch 4/1000
83/83 [==============================] - 0s 2ms/step - loss: 0.3551 - val_loss: 0.3524
Epoch 5/1000
83/83 [==============================] - 0s 3ms/step - loss: 0.3321 - val_loss: 0.3293
Epoch 6/1000
83/83 [==============================] - 0s 3ms/step - loss: 0.3087 - val_loss: 0.3227
Epoch 7/1000
83/83 [==============================] - 0s 3ms/step - loss: 0.2799 - val_loss: 0.2717
Epoch 8/1000
83/83 [==============================] - 0s 3ms/step - loss: 0.2471 - val_loss: 0.2828
Epoch 9/1000
83/83 [==============================] - 0s 3ms/step - loss: 0.2288 - val_loss: 0.2699
Epoch 10/1000
83/83 [==============================] - 0s 2ms/step - loss: 0.2065 - val_loss: 0.1874

In [ ]:
current_state = [4,3]
current_state = np.array(current_state, dtype=np.float32)
predicted_reward_current = reward_predictor_3states_eval.predict(np.expand_dims(current_state, axis=0))
print("Predicted Reward Current State:", predicted_reward_current[0, 0])

1/1 [==============================] - 0s 33ms/step
Predicted Reward Current State: 0.00042444468


State, action, Next State -> Reward

In [ ]:

# import tensorflow as tf
# import numpy as np

# # Step 1: Prepare the data
# behavior_policies = [...]  # Replace [...] with your actual behavior_policies list

# # Initialize lists to store all 'current_state', 'action', 'next_state', and 'reward' values
# all_current_states = []
# all_actions = []
# all_next_states = []
# all_rewards = []

# # Extract the 'current_state', 'action', 'next_state', and 'reward' from the 'behavior_policies' list
# for trajectory in behavior_policies:
#     for state, action, reward, next_state in trajectory:
#         all_current_states.append(state)
#         all_actions.append(action)
#         all_next_states.append(next_state)
#         all_rewards.append(reward)

# # Convert 'current_states', 'actions', 'next_states', and 'rewards' into appropriate formats for training
# all_current_states = np.array(all_current_states, dtype=np.float32)
# all_actions = np.array(all_actions, dtype=np.float32)
# all_next_states = np.array(all_next_states, dtype=np.float32)
# all_rewards = np.array(all_rewards, dtype=np.float32)

## State + proximity to good/bad regions -> Cumulative Reward

# OPE Calculations

## Importance Weights

In [41]:
eval_policy = {"up": 0.4, "down": 0.1, "left": 0.1, "right": 0.4}
behav_policy = {"up": 0.25, "down": 0.25, "left": 0.25, "right": 0.25}
def calculate_importance_weights(eval_policy, behav_policy, behavior_policies):
  all_weights = []
  for trajectory in behavior_policies:
    cum_ratio = 1
    cumul_weights = []
    for step in trajectory:
        ratio = eval_policy[step[1]]/behav_policy[step[1]]
        # print("Ratio:",ratio)
        cum_ratio *= ratio
        cumul_weights.append(cum_ratio)
        # print("Cumul:",cum_ratio)
    all_weights.append(cumul_weights)

  return all_weights

## IS

In [43]:
def per_step_IS(behavior_policies, phi_trajectories = 0.3):
  all_timesteps = []
  gamma = 0.9
  policy_for_scope,_ = subset_policies(behavior_policies, phi_trajectories)
  scope_weights = calculate_importance_weights(eval_policy, behav_policy, policy_for_scope)
  for j in range(len(scope_weights)):
    Timestep_values = []
    for i in range(len(scope_weights[j])-1):
      timestep = gamma**(i)*scope_weights[j][i]*policy_for_scope[j][i][2]
      # print("Timestep: ",timestep)
      Timestep_values.append(timestep)

    all_timesteps.append(Timestep_values)

  V_per_traj = [sum(sublist) for sublist in all_timesteps]

  seed_value = 42
  np.random.seed(seed_value)

  num_trajectories_to_sample = max(1, len(V_per_traj))

  bootstrap_samples = [np.random.choice(V_per_traj, size=num_trajectories_to_sample, replace=True)
                        for _ in range(100)]

  V_per_sample = [sum(sample)/len(bootstrap_samples) for sample in bootstrap_samples]
  V_per_sample = np.array(V_per_sample)

  std_deviation = np.std(V_per_sample)
  quartiles = np.percentile(V_per_sample, [25, 50, 75])
  max_value = np.max(V_per_sample)
  min_value = np.min(V_per_sample)

  return V_per_sample, std_deviation, quartiles, max_value, min_value

In [44]:
# IS_per_traj, is_std, is_quartiles, is_max_value, is_min_value = per_step_IS(behavior_policies,0.3)
# print(is_std)
# print(is_quartiles)
# print(is_max_value)
# print(is_min_value)

0.10933610817754
[-0.84036769 -0.77909955 -0.69050267]
-0.5479659789843923
-1.0898425977186181


## SCOPE

In [36]:
import numpy as np
def SCOPE(behavior_policies, beta, phi_trajectories = 0.3):
    all_timesteps = []
    gamma = 0.9
    policy_for_scope,_ = subset_policies(behavior_policies, phi_trajectories)
    scope_weights = calculate_importance_weights(eval_policy, behav_policy, policy_for_scope)
    for j in range(len(scope_weights)):
        Timestep_values = []
        for i in range(len(scope_weights[j]) - 1):
            features = policy_for_scope[j][i][5] + policy_for_scope[j][i][6]
            features_next = policy_for_scope[j][i + 1][5] + policy_for_scope[j][i + 1][6]
            timestep = gamma ** (i) * scope_weights[j][i] * (policy_for_scope[j][i][2] + gamma * phi(features_next, beta) - phi(features, beta))
            Timestep_values.append(timestep)

        all_timesteps.append(Timestep_values)

    V_per_traj = [sum(sublist) for sublist in all_timesteps]



    seed_value = 42
    np.random.seed(seed_value)

    num_trajectories_to_sample = max(1, len(V_per_traj))

    bootstrap_samples = [np.random.choice(V_per_traj, size=num_trajectories_to_sample, replace=True)
                         for _ in range(100)]

    V_per_sample = [sum(sample)/len(bootstrap_samples) for sample in bootstrap_samples]
    V_per_sample = np.array(V_per_sample)

    std_deviation = np.std(V_per_sample)
    quartiles = np.percentile(V_per_sample, [25, 50, 75])
    max_value = np.max(V_per_sample)
    min_value = np.min(V_per_sample)

    return V_per_sample, std_deviation, quartiles, max_value, min_value
    # return bootstrap_samples


In [112]:
samples = SCOPE(behavior_policies, beta, 0.3)

In [120]:
sum(samples[99])

-147.58609119391252

In [132]:
# # all_weights = calculate_importance_weights(eval_policy, behav_policy, behavior_policies)
# beta =  [ 0.2609209,   0.47456879, -0.52815694]
# # beta = [0,0,0]
# V_per_sample, scope_std, scope_quartiles, scope_max_value, scope_min_value = SCOPE(behavior_policies,beta,0.3)
# print(scope_std)
# print(scope_quartiles)
# print(scope_max_value)
# print(scope_min_value)

0.15306293298484386
[-1.2602556  -1.17031182 -1.07600677]
-0.8297692802152171
-1.662222105105361


# Variance Preparation and Calculation

In [34]:
def phi(features, beta):
  features = np.array(features)
  beta = np.array(beta)
  phi_s = np.dot(beta,features)
  return phi_s


In [38]:
import random
# gamma = 0.9
# beta = [random.random() for _ in range(3)]
def variance_terms(policy_set,gamma, beta):
  all_weights = calculate_importance_weights(eval_policy, behav_policy, policy_set)
  y_w_r_all = 0
  r_all = 0
  f_a = 0
  for j in range(len(policy_set)):
    y_w_r = 0
    r = 0
    for i in range(len(policy_set[j])):
      features = policy_set[j][i][5]+policy_set[j][i][6]
      y_w_r += gamma**(i)*all_weights[j][i]*policy_set[j][i][2]
      if i>0 & i<len(policy_set):
        r += phi(features, beta)*(all_weights[j][i-1]-all_weights[j][i])
    y_w_r_all += y_w_r
    f_a +=  gamma**(len(policy_set[j]))*all_weights[j][-1]*phi(features,beta) - phi(features, beta) # fix the features part
    r_all += r

  IS = y_w_r_all/len(policy_set)
  R = r_all/len(policy_set)
  F = f_a/len(policy_set)
  return IS, R, F


In [39]:
def subset_policies(policy, percent_to_estimate_phi):
    seed_value = 42
    np.random.seed(seed_value)
    num_policies = len(policy)
    num_policies_to_estimate_phi = int(num_policies * percent_to_estimate_phi)

    policy_for_scope = policy[num_policies_to_estimate_phi:]
    policy_for_phi = policy[:num_policies_to_estimate_phi]

    return policy_for_scope, policy_for_phi

def calc_variance(behavior_policies, gamma, beta, num_bootstrap_samples = 100,phi_trajectories = 0.3):
  # Set the seed value (you can use any integer value)
  seed_value = 42
  np.random.seed(seed_value)
  num_trajectories_to_sample = max(1, int(len(behavior_policies) * phi_trajectories))

  policy_for_scope, policy_for_phi = subset_policies(behavior_policies, percent_to_estimate_phi=phi_trajectories)
  num_trajectories_to_sample = max(1, len(policy_for_phi))

  bootstrap_samples = [np.random.choice(policy_for_phi, size=num_trajectories_to_sample, replace=True)
                         for _ in range(num_bootstrap_samples)]
  IS_all = []
  R_all = []
  F_all = []

  for pol in bootstrap_samples:
    IS, R, F = variance_terms(pol,0.9,beta)
    IS_all.append(IS)
    R_all.append(R)
    F_all.append(F)
  IS_sq = np.mean([num**2 for num in IS_all])
  IS_R_F = 2*np.mean([IS_all[i]*(R_all[i]+F_all[i]) for i in range(len(IS_all))])
  R_sq = np.mean([num**2 for num in R_all])
  IS_sq_all = (np.mean(IS_all))**2
  IS_r_t_f = 2*np.mean(IS_all)*np.mean([R_all[i]+F_all[i] for i in range(len(R_all))])
  R_sq_all = (np.mean(R_all))**2

  variance_scope = IS_sq + IS_R_F + R_sq - IS_sq_all - IS_r_t_f - R_sq_all
  variance_is = IS_sq - IS_sq_all
  return variance_scope, variance_is

An example of an initial guess of phi can be seen below, as you can see the SCOPE variance is not ideal.

In [57]:
scope_set, phi_set = subset_policies(behavior_policies, 0.3)
variance_scope, variance_is = calc_variance(phi_set,0.9,[-0.1,.1,.1], 100, 0.3)
print("Var SCOPE: ",variance_scope)
print("Var IS: ",variance_is)
print("Percent change in variance: ",((variance_scope-variance_is)/variance_is)*100)

<ipython-input-39-f5a21e0b065d>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bootstrap_samples = [np.random.choice(policy_for_phi, size=num_trajectories_to_sample, replace=True)


Var SCOPE:  0.7292656998003821
Var IS:  0.3796743263659492
Percent change in variance:  92.07664283770379


# Optimization

Here we aim to optimize beta to minimize SCOPE variance.

In [55]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function to minimize variance_scope
def objective_function(beta, behavior_policies, phi_trajectories):
    scope_set, phi_set = subset_policies(behavior_policies, phi_trajectories)
    variance_scope, variance_is = calc_variance(phi_set, 0.9, beta, 100, 0.3)
    return variance_scope

# Set the initial values of beta
initial_beta = np.array([ 0.2610704,   0.30396575, -0.43850237])


def optimize_variance_scope(initial_beta, behavior_policies, phi_trajectories):
    # Lists to store beta and variance_scope values at each iteration
    all_betas = []
    all_variance_scopes = []

    # Callback function to record beta and variance_scope values at each iteration
    def callback_function(beta):
        all_betas.append(beta.copy())
        variance_scope = objective_function(beta, behavior_policies, phi_trajectories)
        all_variance_scopes.append(variance_scope)
        print("Iteration:", len(all_betas))
        print("Beta:", beta)
        print("Variance Scope:", variance_scope)
        print("----------")

    # Run the optimization with the callback
    result = minimize(
        objective_function,
        initial_beta,
        args=(behavior_policies, phi_trajectories),
        method='L-BFGS-B',
        callback=callback_function
    )

    # Extract the optimal beta values
    optimal_beta = result.x

    return optimal_beta, all_betas, all_variance_scopes


In [56]:
optimal_beta, _, _ = optimize_variance_scope(initial_beta, behavior_policies, 0.3)

<ipython-input-39-f5a21e0b065d>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bootstrap_samples = [np.random.choice(policy_for_phi, size=num_trajectories_to_sample, replace=True)


Iteration: 1
Beta: [ 0.16821737  0.35271354 -0.42156545]
Variance Scope: 0.11317799047435423
----------
Iteration: 2
Beta: [ 0.15853249  0.34137881 -0.42710298]
Variance Scope: 0.11040088606596254
----------
Iteration: 3
Beta: [ 0.15073224  0.32968859 -0.42471605]
Variance Scope: 0.1093933569252897
----------
Iteration: 4
Beta: [ 0.14615259  0.3202478  -0.4164246 ]
Variance Scope: 0.10883472336459371
----------
Iteration: 5
Beta: [ 0.14331703  0.31081431 -0.40169853]
Variance Scope: 0.108436847289928
----------
Iteration: 6
Beta: [ 0.14352606  0.31073858 -0.40072254]
Variance Scope: 0.10843321214844745
----------
Iteration: 7
Beta: [ 0.14356653  0.31079481 -0.40072313]
Variance Scope: 0.10843319408861363
----------
Iteration: 8
Beta: [ 0.14356994  0.31080162 -0.40072872]
Variance Scope: 0.10843319398401882
----------


# Value estimates of IS and SCOPE estimators

In [50]:
# all_weights = calculate_importance_weights(eval_policy, behav_policy, behavior_policies)
beta =  [ 0.2609209,   0.47456879, -0.52815694]
# beta = [0,0,0]
V_per_sample, scope_std, scope_quartiles, scope_max_value, scope_min_value = SCOPE(behavior_policies,optimal_beta,0.3)
print("SCOPE Std Dev: ", scope_std)
print("SCOPE quartiles: ",scope_quartiles)
print("SCOPE max: ",scope_max_value)
print("SCOPE min",scope_min_value)

SCOPE Std Dev:  0.10569617855732055
SCOPE quartiles:  [-0.59477096 -0.52791441 -0.44771446]
SCOPE max:  -0.2758543575915438
SCOPE min -0.8459066085675662


In [51]:
IS_per_traj, is_std, is_quartiles, is_max_value, is_min_value = per_step_IS(behavior_policies,0.3)
print("IS std dev: ",is_std)
print("IS quartiles: ",is_quartiles)
print("IS max: ",is_max_value)
print("IS min: ", is_min_value)

IS std dev:  0.10933610817754
IS quartiles:  [-0.84036769 -0.77909955 -0.69050267]
IS max:  -0.5479659789843923
IS min:  -1.0898425977186181
